# SCI6485 Introduction to Generative Artificial Intelligence

##  STYLEGAN3


**Harvard University**<br/>
**Fall 2023**<br/>
**Instructor:**  Sabrina Osmany<br/>
**TA:**  Jiabin Wei<br/>

**DISCLAIMER**: No public reproduction of this code is allowed without the explicit consent of their authors.

<hr style="height:2pt">

---

 This tutorial is heavily borrowed from this github [repo](https://github.com/derekphilipau/machinelearningforartists).

# Training custom data with StyleGAN

In this tutorial, we only want to familiarize ourselves with running StyleGAN (in this case, the newest version) and seeing how training progresses.

## Verify Runtime is GPU

In the menu, select Runtime -> Change Runtime Type and verify you are using the **GPU**.  Also select **High-RAM** if you are using Colab Pro.

The `nvidia-smi` command below should **NOT** display *"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver."*

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-91f6ad27-eed8-5ae4-4457-5c095df93224)


## Mount your Google Drive

You will be storing the training models and progress images on your Google Drive.  This is very convenient for viewing progress, and if your Colab notebook is disconnected you will not lose your models.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Stylegan pytorch Prerequisites

In [3]:
import torch
import torchvision
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

Selected device: cuda
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.9/205.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.4.9
    Uninstalling imageio-ffmpeg-0.4.9:
      Successfully uninstalled imageio-ffmpeg-0.4.9


## Get the StyleGAN code

In [4]:
# clone StyleGan repo from github
!git clone https://github.com/NVlabs/stylegan3

Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 212 (delta 0), reused 1 (delta 0), pack-reused 207
Receiving objects: 100% (212/212), 4.17 MiB | 14.53 MiB/s, done.
Resolving deltas: 100% (99/99), done.


 create a dataset folder under your stylegan directory

In [5]:
mkdir /content/stylegan3/datasets

## Load a custom dataset

In [6]:
#######################################################
#      unzip the custome dataset zip file to the 'datasets' folder under the 'stylegan3' directory
#      you need to replace the path after 'unzip' with your zip file directory
#######################################################

# Custom datasets can be created from a folder containing images
# unzip your custom data zip file to stylegan3 dataset directory
!unzip /content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data/custom_data.zip -d /content/stylegan3/datasets

Streaming output truncated to the last 5000 lines.
  inflating: /content/stylegan3/datasets/custom_data/emerald-85.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data/._emerald-85.png  
  inflating: /content/stylegan3/datasets/custom_data/crystal-140.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data/._crystal-140.png  
  inflating: /content/stylegan3/datasets/custom_data/platinum-104.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data/._platinum-104.png  
  inflating: /content/stylegan3/datasets/custom_data/heartgold-soulsilver-485.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data/._heartgold-soulsilver-485.png  
  inflating: /content/stylegan3/datasets/custom_data/heartgold-soulsilver-491.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data/._heartgold-soulsilver-491.png  
  inflating: /content/stylegan3/datasets/custom_data/silver-241.png  
  inflating: /content/stylegan3/datasets/__MACOSX/custom_data/

In [7]:
# remove the "__MACOSX" directory from a Unix-based file system, typically found in ZIP or compressed archives created on macOS.
!rm -rf /content/stylegan3/datasets/__MACOSX

In [8]:
cd /content/stylegan3

/content/stylegan3


In [9]:
ls

avg_spectra.py   dnnlib/          gen_images.py  LICENSE.txt   training/
calc_metrics.py  Dockerfile       gen_video.py   metrics/      train.py
datasets/        docs/            gui_utils/     README.md     visualizer.py
dataset_tool.py  environment.yml  legacy.py      torch_utils/  viz/


In [10]:
!pwd

/content/stylegan3


## Tips for training a StyleGAN3 with limited computing resources
Generally, there are two ways to do it:

I.  Make the training process more efficient <br>
II. Make your custom dataset easier to train with <br>
III. Resume the training with the latest training result<br>
IV. Train your data with a pre-trained model (transfer learning)<br><br>


**I. Make the training process faster**
  * Reduce the resolution of images:

  use smaller image size to reduce the number of trainable parameters
  ```python
  # you can also scale down the resolution of images here. You could use this line to check the explanation for the parameters: !python dataset_tool.py --help
  !python dataset_tool.py --source=./datasets/custom_data_2 --dest=./datasets/customdataset.zip --resolution=16x16
  ```
  However, as the latent space dimension of the generator of the StyleGAN3 is fixed, reducing the resolution of your custom dataset only reduce the trainable parameter size in the discriminator in this model.

  If you want to further reduce the trainiable parameters, you can use a less complex StyleGAN model. For example, Mark used another StyleGAN model in this [repo](https://github.com/TAUIL-Abd-Elilah/Generative-Adversarial-Networks--GANs/blob/main/StyleGAN%20implementation%20from%20scratch%20PyTorch.ipynb). It seems to be simpler than the StyleGAN3 used in this colab.

  Although it may not very easy to change the model architecture, it is possile to make some minor changes to reduce the trainable parameters. For example,  we can use the ``cmap`` parameter to reduce the maximum channel size. Or use the ``map-depth`` parameter to set the number of mapping network layers, but we use the stylegan3-t model configuration in this colab, which only has 2 mapping network layers. You will find that ``cmap`` can greatly reduce the trainable parameters and the training time.

  ```python
  # train the model. change the batch size to a larger number but you should make sure that it won't
  !python train.py --outdir=... --cfg=stylegan3-t --data=...  --gpus=1 --batch=32 --batch-gpu=4  --map-depth=2 --cmap=256 ...
  ```
  (Mapping network: is an essential component that takes a random noise vector as input and maps it to an intermediate latent space representation that controls the appearance of the generated images. The purpose of the mapping network is to learn a transformation that can shape the latent space in a way that makes it more amenable for generating high-quality images. This transformation helps disentangle different aspects of image generation, such as controlling factors like facial features, pose, and style. It separates the stochastic part from the deterministic part of image generation.)  <br><br>




  * Mildly increase the image batch size:

  When you train a deep learning model, you typically have a dataset consisting of a large number of training examples. The batch size specifies how many of these examples are loaded into memory at a time. For example, if you set the batch size to 32, the training process will load and process 32 examples at once in parallel. So if we increase the batch size  a little bit:

    - Efficient GPU Utilization: With larger batch sizes, you can better utilize the parallel processing capabilities of modern GPUs, potentially speeding up training.
    - Stable Gradients: Larger batch sizes can provide more stable and accurate gradient estimates, which can lead to more robust training.

  ```python
  # train the model. change the batch size to a larger number but you should make sure that it won't
  !python train.py --outdir=... --cfg=stylegan3-t --data=...  --gpus=1 --batch=32 --batch-gpu=4...
  ```

  However, the batch size directly affects the GPU memory usage. Each example in the batch is stored in memory, and the intermediate activations and gradients are also stored for the purpose of backpropagation. The GPU processes the entire batch in parallel. The size of the batch must be small enough to fit within the available GPU memory. If the batch size is too large for the GPU to handle, you may encounter out-of-memory errors. <br><br>

**II. Make your custom dataset easier to work with**

  * Consistent Object or Scene Types:
  
  Custom datasets that focus on a specific object or scene type can be easier for GANs to generate. For instance, a dataset of cats, dogs, or specific objects is generally easier to work with than a diverse collection of unrelated items.

  * Large Inter-class Variability:
  
  GANs can handle multiple classes, but you would better to make sure that the differences between different classes or categories are clear. For example, if you have a dataset of cars and bikes, the distinct differences in shape and appearance make it easier for GANs to generate these objects.
  
  * Few Background Variations:
  
  If the background of the images in your dataset remains relatively consistent, GANs can focus more on modeling the primary object or content.

  * Consistent Lighting and Camera Conditions:
  
  Datasets with consistent lighting and camera conditions are generally easier to work with. Changes in lighting, viewpoint, or camera conditions can introduce additional challenges for GANs.<br><br>

**III. Resume the training with the latest training result**

  * The StyleGAN3 model we use here will automatically save your training result every n ticks. The default n is 50 and you could set this n in with the ``snap`` parameter. You can also change the size of tick with the ``tick`` parameter. The default tick size is 4, which means that every 4k images, the model will save the snapshot.

  ```python
  #######################################################
  #      set --outdir= as the directory you want to store your training result
  #      set --data= as the compressed zip file generated by the dataset_tool.py from above
  #######################################################

  !python train.py --outdir=/content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_result_test --cfg=stylegan3-t --data=./datasets/customdataset.zip  --gpus=1 --batch=32 --batch-gpu=4 --gamma=10.0 --mirror=1 --kimg=5000 --snap=1
  ```

  To resume the training process, you need to tell the train.py where the latest training result pkl is. You need modify value of the `resume` parameter in this code cell and put your latest pkl file directory here。
  
  ```python
  #######################################################
  #      optional: fine tune a pretrained model
  #      but you need to resize the images to fit the pre-trained model
  #######################################################

  !python train.py --outdir=/content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_result --cfg=stylegan3-t --data=./datasets/customdataset.zip  --gpus=1 --batch=32 --batch-gpu=4 --gamma=10.0 --mirror=1 --kimg=5000 --snap=1 --resume=content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_2/00000-stylegan3-t-customdataset-gpus1-batch32-gamma10/network-snapshot-000012.pkl
  ```

  (Tick is another way to track the progress of the training process. For example, you may see information like "1M ticks" or "10M ticks," where "M" stands for million, indicating that a certain number of training images have been completed. We say one training image is completed when it goes through a single forward and backward pass through the network, and finish computing gradients and updating parameters.)<br><br>
  

**IV. Train your data with a pre-trained model (transfer learning)**
  
  
   * This suggesion is copied from the StyleGAN3 official [configuration doc](https://github.com/NVlabs/stylegan3/blob/main/docs/configs.md) :
  
   Transfer learning makes it possible to reach very good results very quickly, especially when the training set is small and/or the images resemble the ones produced by a pre-trained model. To enable transfer learning, you can point --resume to one of the pre-trained models that we provide for StyleGAN3 and StyleGAN2. For example:

  ```python
  # Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
  python train.py --outdir=~/training-runs --cfg=stylegan3-r --data=~/datasets/metfacesu-1024x1024.zip \
    --gpus=8 --batch=32 --gamma=6.6 --mirror=1 --kimg=5000 --snap=5 \
    --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl
  ```

  The pre-trained model should be selected to match the specified config, resolution, and architecture-related hyperparameters (e.g., --cbase, --map-depth, and --mbstd-group). You check this by looking at the fakes_init.png exported by train.py at the beginning; if the configuration is correct, the images should look reasonable.
  
  With transfer learning, the results may be improved slightly by adjusting ``--freezed``, in addition to the above guidelines for ``--gamma``, ``--mirror``, and ``--aug``. In our experience, ``--freezed=10`` and ``--freezed=13`` tend to work reasonably well.

## Prepare your custom dataset for use by StyleGAN

In [11]:
#######################################################
#      set --source= as the directory your dataset folder
#      set --dest= as the directory and the folder name of the new compressed zip file
#######################################################

# Datasets are stored as uncompressed ZIP archives containing uncompressed PNG files and a metadata file dataset.json for labels.
# !python dataset_tool.py --source=./datasets/custom_data --dest=./datasets/customdataset.zip

# you can also scale down the resolution of images here. You could use this line to check the explanation for the parameters: !python dataset_tool.py --help
# !python dataset_tool.py --source=./datasets/custom_data_2 --dest=./datasets/customdataset.zip --resolution=32x32
!python dataset_tool.py --source=./datasets/custom_data --dest=./datasets/customdataset.zip --resolution=64x64

100% 4729/4729 [00:12<00:00, 376.21it/s]


## Create Folders on Your Google Drive to Store Results

If we accidentally close our browser or the Colab runtime disconnects, we will lose all of our training models and progress images.  Therefore we want to store the training data on our Google Drive.  The following cells will help you create a new folder on your Google Drive and then your training data will be stored in this path later

In [ ]:
!mkdir /content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_result_test_2

If you get an error "cannot create directory", it's probably because the folder already exists on your Google Drive and you can ignore the error.

## Train from Scratch

This cell block will train StyleGAN from scratch.  Training from scratch is much slower than using *transfer learning* on a previously trained model.  However, the purpose of this tutorial is to a) try training for the first time and b) notice how the progress images develop over time.

In [ ]:
# if you want check the meaning of the parameters in the train.py, you could run this code
!python train.py --help

Usage: train.py [OPTIONS]

  Train a GAN using the techniques described in the paper "Alias-Free
  Generative Adversarial Networks".

  Examples:

  # Train StyleGAN3-T for AFHQv2 using 8 GPUs.
  python train.py --outdir=~/training-runs --cfg=stylegan3-t --data=~/datasets/afhqv2-512x512.zip \
      --gpus=8 --batch=32 --gamma=8.2 --mirror=1

  # Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
  python train.py --outdir=~/training-runs --cfg=stylegan3-r --data=~/datasets/metfacesu-1024x1024.zip \
      --gpus=8 --batch=32 --gamma=6.6 --mirror=1 --kimg=5000 --snap=5 \
      --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl

  # Train StyleGAN2 for FFHQ at 1024x1024 resolution using 8 GPUs.
  python train.py --outdir=~/training-runs --cfg=stylegan2 --data=~/datasets/ffhq-1024x1024.zip \
      --gpus=8 --batch=32 --gamma=10 --mirror=1 --aug=noaug

Options:
  --outdir DIR   

In [ ]:
#######################################################
#      set --outdir= as the directory you want to store your training result
#      set --data= as the compressed zip file generated by the dataset_tool.py from above
#######################################################

# train the model
!python train.py --outdir=../drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_result_test_2 --cfg=stylegan3-r --data=./datasets/customdataset.zip --cmax=256 --gpus=1 --batch=32 --gamma=2 --batch-gpu=16 --snap=1 --workers=2


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 65536,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 256
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 2.0,
    "blur_init_sig

In [ ]:
#######################################################
#      optional: fine tune a pretrained model
#                but you need to resize the images to fit the pre-trained model
#######################################################

# you could also fine a StyleGan3: Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
!python train.py --outdir=../drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_result_test_2 --cfg=stylegan3-r --data=./datasets/customdataset.zip --cmax=256 --gpus=1 --batch=32 --gamma=2 --batch-gpu=16 --snap=1 --workers=2 \
--resume=../drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_result_test_2/00000-stylegan3-r-customdataset-gpus1-batch32-gamma2/network-snapshot-000012.pkl


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 65536,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 256
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 2.0,
    "blur_init_sig

## Wait an Hour or Two

Normal training of a dataset can take days.  For this demonstration, we only want to train for a few hours in order to get familiar with the training process and see the initial results.

Results are stored in your Google Drive under your custom path defined above.

Each training run is stored in a separate directory.  The initial training run is stored in `00000-customdataset...`.  If you run training twice, the second run will be stored in `00001-customdataset...`.  And so on.

Inside the training run directory you will see various files.  `reals.png` shows a sample of the training dataset.  You should see various your custom images in this image. `fakes000000.png` is a sample of generated images from the initial model.  `network-snapshot-XXXXXX.pkl` is the actual model which can be used later to generate "fake" images, videos, etc.

As training progresses you will see more `fakes` & `network-snapshot` files created.

Since we only want to test a training, once you get to around the 500th iteration, you can stop the above training cell.  Then take a look at the `fakes` images and familiarize yourself with how the training progresses.



## Generate images

In [ ]:
#######################################################
#      set the directory where you want to store the new generated images
#######################################################

# make a directory to store newly generated images
!mkdir /content/drive/MyDrive/MDE/SCI6485/StyleGan/generated_images_2

mkdir: cannot create directory ‘/content/drive/MyDrive/MDE/SCI6485/StyleGan/generated_images’: File exists


In [ ]:
#######################################################
#      set --outdir= as the path above
#      set --network= as the training result snapshot you want to use for generating the image
#######################################################

# load the training result and generate a new image
# the new images will be stored in the path you created above
!python gen_images.py --outdir=/content/drive/MyDrive/MDE/SCI6485/StyleGan/generated_images_2 --trunc=1 --seeds=2 \
    --network=/content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data_2/00000-stylegan3-t-customdataset-gpus1-batch32-gamma10/network-snapshot-000012.pkl

Loading networks from "/content/drive/MyDrive/MDE/SCI6485/StyleGan/custom_data/00000-stylegan3-t-pokemondataset-gpus1-batch32-gamma10/network-snapshot-000012.pkl"...
Generating image for seed 2 (0/1) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
